In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 7.2 MB/s 
     |████████████████████████████████| 7.6 MB 36.0 MB/s 
     |████████████████████████████████| 163 kB 45.9 MB/s 


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 7.2 MB/s 
     |████████████████████████████████| 212 kB 53.5 MB/s 
     |████████████████████████████████| 115 kB 57.1 MB/s 
     |████████████████████████████████| 127 kB 24.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 7.8 MB/s 


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np
import torch
import nltk
import re
import string
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [44]:
text = '''
Das: Hi and welcome to the a16z podcast. I’m Das, and in this episode, I talk SaaS go-to-market with David Ulevitch and our newest enterprise general partner Kristina Shen. The first half of the podcast looks at how remote work impacts the SaaS go-to-market and what the smartest founders are doing to survive the current crisis. The second half covers pricing approaches and strategy, including how to think about free versus paid trials and navigating the transition to larger accounts. But we start with why it’s easier to move upmarket than down… and the advantage that gives a SaaS startup against incumbents.
David: If you have a cohort of customers that are paying you $10,000 a year for your product, you’re going to find a customer that self-selects and is willing to pay $100,000 a year. Once you get one of those, your organization will figure out how you sell to, how you satisfy and support, customers at that price point and that size. But it’s really hard for a company that sells up market to move down market, because they’ve already baked in all that expensive, heavy lifting sales motion. And so as you go down market with a lower price point, usually, you can’t actually support it.
Das: Does that mean that it’s easier for a company to do this go-to-market if they’re a new startup as opposed to if they’re a pre-existing SaaS?
Kristina: It’s culturally very, very hard to give a product away for free that you’re already charging for. It feels like you’re eating away at your own potential revenue when you do it. So most people who try it end up pulling back very quickly.
David: This is actually one of the key reasons why the bottoms up SaaS motion is just so competitive, and compelling, and so destructive against the traditional sales-driven test motion. If you have that great product and people are choosing to use it, it’s very hard for somebody with a sales-driven motion, and all the cost that’s loaded into that, to be able to compete against it. There are so many markets where initially, we would look at companies and say, “Oh, well, this couldn’t possibly be bottoms up. It has to be sold to the CIO. It has to be sold to the CSO or the CFO.” But in almost every case we’ve been wrong, and there has been a bottoms up motion. The canonical example is Slack. It’s crazy that Slack is a bottoms up company, because you’re talking about corporate messaging, and how could you ever have a messaging solution that only a few people might be using, that only a team might be using? But now it’s just, “Oh, yeah, some people started using it, and then more people started using it, and then everyone had Slack.”
Kristina: I think another classic example is Dropbox versus Box. Both started as bottoms up businesses, try before you buy. But Box quickly found, “Hey, I’d rather sell to IT.” And Dropbox said, “Hey, we’ve got a great freemium motion going.” And they catalyzed their business around referrals and giving away free storage and shared storage in a way that really helped drive their bottoms up business.
Das: It’s a big leap to go from selling to smaller customers to larger customers. How have you seen SaaS companies know or get the timing right on that? Especially since it does seem like that’s really related to scaling your sales force?
Kristina: Don’t try to go from a 100-person company to a 20,000-person company. Start targeting early adopters, maybe they’re late stage pre-IPO companies, then newly IPO’d companies. Starting in tech tends to be a little bit easier because they tend to be early adopters. Going vertical by vertical can be a great strategy as well. Targeting one customer who might be branded in that space, can help brand yourself in that category. And then all their competitors will also want your product if you do a good job. A lot of times people will dedicate a sales rep to each vertical, so that they become really, really knowledgeable in that space, and also build their own brand and reputation and know who are the right customers to target.
Das: So right now, you’ve got a lot more people working remote. Does this move to remote work mean that on-premise software is dying? And is it accelerating the move to software as a service?
Kristina: This remote work and working from home is only going to catalyze more of the conversion from on-premise over to cloud and SaaS. In general, software spend declines 20% during an economic downturn. This happened in ’08, this happened in ’01. But when we look at the last downturn in ’08, SaaS spend actually, for public companies, increased, on average, 10%, which means there’s a 30% spread, which really shows us that there was a huge catalyst from people moving on-premise to SaaS.
David: And as people work remote, the ability to use SaaS tools is much easier than having to VPN back into your corporate network. We’ve been seeing that, inside sales teams have been doing larger and larger deals, essentially moving up market on the inside, without having to engage with field sales teams. In fact, a lot of the new SaaS companies today rather than building out a field team, they have a hybrid team, where people are working and closing deals on the inside and if they had to go out and meet with a customer, they would do that. But by and large, most of it was happening over the phone, over email, and over videoconferencing. And all the deals now, by definition, are gonna be done remote because people can’t go visit their customers in person.
Das: So with bottoms up, did user behavior and buyer behavior change, so the go-to-market evolved? Or did the go-to-market evolve and then you saw user and buyer behavior change? I’m curious with this move to remote work. Is that going to trigger more changes or has the go-to-market enabled that change in user behavior, even though we see that change coming because of a lot of forces outside of the market?
Kristina: I definitely think they are interrelated. But I do think it was a user change that catalyzed everything. We decided that we preferred better software, and we tried a couple products. We were able to purchase off our credit card. And then IT and procurement eventually said, “Wow, everyone’s buying these already, I might as well get a company license and a company deal so I’m not paying as much.” While obviously software vendors had to offer the products that could be self-served, users started to realize they had the power, they wanted to use better software, they paid with their credit cards. And now software vendors are forced to change their go-to-market to actually suit that use case.
Das: If that’s the case that when user behavior has changed, it’s tended to be the catalyzing force of bigger changes in the go-to-market, what are some of the changes you foresee for SaaS because the world has changed to this new reality of remote work and more distributed teams?
David: We’re in a very uncertain economic environment right now. And a couple of things will become very clear over the next 3 to 9 to 15 months — you’re going to find out which SaaS products are absolutely essential to helping a business operate and run, and which ones were just nice to have and may not get renewed. I think on the customer, buying side, you’re very likely to see people push back on big annual commitments and prefer to go month-to-month where they can. Or you’ll see more incentives from SaaS startups to offer discounts for annual contracts. You’re going to see people that might sign an annual contract, but they may not want to pay upfront. They may prefer to meter the cash out ratably over the term of the contract. And as companies had empowered and allowed budget authority to be pushed down in organizations, you’re gonna see that budget authority get pulled back, more scrutiny on spending, and likely a lot of SaaS products not get renewed that turned out to not be essential.
Kristina: I think the smartest founders are making sure they have the runway to continue to exist. And they’re doing that in a couple of ways. They’re preserving cash, and they are making sure that their existing customers are super, super happy, because retaining your customers is so important in this environment. And they’re making sure that they have efficient or profitable customer acquisition. Don’t spend valuable dollars acquiring customers. But acquire customers efficiently that will add to a great existing customer base.
Das: To go into pricing and packaging for SaaS for a moment, what are some of the different pricing approaches that you see SaaS companies taking?
Kristina: The old school way of doing SaaS go-to-market is bundle everything together, make the pricing super complex, so you don’t actually understand what you’re paying for. You’re forced to purchase it because you need one component of the product. New modern SaaS pricing is keep it simple, keep it tied to value, and make sure you’re solving one thing really, really well.
David: You want to make it easy for your customers to give you money. And if your customers don’t understand your pricing, that’s a huge red flag. Sometimes founders will try to over engineer their pricing model.
Kristina: We talk a lot about everything has to be 10X better than the alternatives. But it’s much easier to be 10X better when you solve one thing very, very well, and then have simple pricing around it. I think the most common that most people know about is PEPM or per employee per month, where you’re charging basically for every single seat. Another really common model is the freemium model. So, think about a Dropbox, or an Asana, or a Skype, where it’s trigger based. You try the product for free, but when you hit a certain amount of storage, or a certain amount of users, then it converts over to paid. And then you also have a time trial, where you get the full experience of the product for some limited time period. And then you’re asked if you want to continue using the product to pay. And then there’s pay as go, and particularly, pay as you go as a usage model. So, Slack will say, “Hey, if your users aren’t actually using the product this month, we won’t actually charge you for it.”
David: The example that Kristina made about Slack and users, everybody understands what a user is, and if they’re using the product, they pay for it, and if they’re not using it, they don’t pay for it. That’s a very friendly way to make it easy for your customers to give you money. If Slack came up with a pricing model that was like based on number of messages, or number of API integration calls, the customer would have no idea what that means.
Kristina: There’s also the consumption model. So Twilio only charges you for every SMS text or phone call that you make on the platform any given month. And so they make money or lose money as your usage goes. The pricing is very aligned to your productivity.
David: Generally, those are for products where the usage only goes in one direction. If you think of a company like Databricks, where they’re charging for storage, or Amazon’s S3 service, it is very aligned with the customer, but it also strategically aligns with the business because they know the switching cost is very high, the churn is very low. And generally, in those businesses, you’re only going to store more data, so they can charge based on usage or volume of data.
Kristina: Recently, there’s been a huge trend of payment as a revenue. It’s particularly common in vertical markets where SaaS companies are adding payments as a revenue in addition to their employee or subscription revenue. If you look at Shopify, for example, more than 50% of their revenue is actually payment revenue. They’re making money every single time you purchase something off one of their shopping cart websites.
Das: When you’re working with a founder or a SaaS startup, how have you seen them find the right pricing model for their product, for their market?
Kristina: Step one is just talk to a lot of customers. Try to figure out what is the market pricing for possible alternatives or competitors, understand their pain points and their willingness to pay. And just throw a price out there, because you have to have a starting point in order to actually test and iterate. Particularly in the SMB, or the bottoms up business, you can test and iterate pretty quickly because you have so many data points.
David: I always tell founders, step one is to just go out there and talk to customers. Step two is just double your prices. I don’t think there’s ever been a great company with a great product that’s fallen apart because their pricing was wrong. But a lot of SaaS startup founders really under price, and you don’t want to find out two or three years later that you were 200% underpriced. A very common thing that SaaS companies do, they’ll have the basic package that either is free or low cost, that you can just sign up online for. They’ll have a middle package where they share some pricing, and then they’ll have the enterprise package where you have to contact sales to find out more. And that way they don’t actually have to show the pricing for that third package. And that gives the salespeople the flexibility to adjust pricing on a per deal basis.
Das: When you’re working with companies, why are they underpricing their products?
David: I think it’s psychological. People need to price on value, and they don’t know how much value they’re delivering relative to “Oh, it only cost me $100 a month to provide this service, so I just need to charge $200.” But if it turns out you’re saving your customer $50,000 a year, then you’re wildly underpriced. You have to remember that SaaS is essentially a proxy for outsourced IT. You’re spending money on a SaaS service to not pay to develop something internally, or to have to pay IT to support something that’s more complex on-prem. Software is much cheaper than people, and so generally, the price point can be much higher.
Kristina: And the other thing is your value increases over time. You’re delivering more features, more products, you understand the customer better. It’s the beauty of the SaaS model and cloud model that you can iterate and push code immediately, and the customer immediately sees value. A lot of times people have the same price point from the first customer sold to three years later and the 200th customer. Quite frankly, you’ve delivered so much value along the way that your price point should have gone up. The other thing I’ll say is a lot of people discount per seat pricing a lot as they move up market. We tend to tell people that the best validation of your product having great product market fit is your ability to hold your price point. So while there is some natural discounting on a per seat basis because people do deserve some volume discounting, I would say try to resist that as much as possible.
Das: Especially for a technical founder, it’s so tempting to get in there and fiddle with these knobs. How do you know when it is time to experiment with your pricing and packaging?
David: If you’re looking at your business and you see that you are doing more deals, and they’re closing faster, you should raise your pricing. And you pay attention to how long it takes to close deals and whether the number of deals is staying consistent as you do that. And, at some point, you’re going to find out when you’re losing deals on price. I think a moment where companies have to plan ahead to avoid having to course correct is after they roll out massive pricing and packaging changes, which are pretty natural as companies move up market. But how they navigate that transition to larger accounts, and how they either bring along or move away from those smaller, earlier customers who got them to where they are, tends to be really important because they can get a lot of noise on Twitter, they can get a lot of blowback from their customers. So Zendesk is a company where they rolled out a major packaging change. And when they rolled it out, they hadn’t planned on grandfathering in their early customers. They got a lot of pushback, and very quickly, they put out a blog post and said, “We hear what you’re saying, we appreciate you building the business that we’ve become today. We do need to have a package for the future. But all the people that have been customers so far will be grandfathered in for at least a period of time into the old model.”
Kristina: If you iterate pricing constantly, you don’t really have this problem because your customers will be used to pricing changes. You normally pair them with new features, and it all kind of works out. But if you have to go through a big grandfather change, I tend to lean towards treating your early customers really, really well. They adopted when you weren’t a big company yet. They probably co-built the product with you in many ways. And so, it’s great to get more dollars out of your customer base, but treat your early customers well.
Das: Are there any other failure modes that you see startups really falling into around pricing and packaging or any common mistakes that they make?
David: I think a lot of founders don’t always map out the cost or model of their pricing and their product relative to their cost of actually doing sales and marketing and customer acquisition.
Kristina: Inside sales is so popular in Silicon Valley. When you’re selling more to an SMB or mid-market type customer, the expectation is that you’re educating and helping the prospective customer over the phone. And so, you’re not expected to be as high touch. But 5K is almost the minimum price point you need to sell to the SMB with an inside sales team in order to pay for the outbound costs and all the conversions, because there is typically a team that sits around the quota carrying rep. And so, price matching — how much your price point is compared to what your go-to-market motion is — matters a lot. Other big failure modes that I see, people guess the ramp time of a sales rep wrong. And ramp time really ties to the segment of customer you’re selling into. It tends be that if you’re selling into the enterprise, the ramp time for sales reps, because sales cycles are so long, tend to be much longer as well. They could be six months plus, could be a year. While if you’re selling more into SMB or mid-market, the ramp time to get a rep up and running can be much shorter, three to six months. Because the sales cycles are shorter, they just iterate much faster, and they ramp up much more quickly.
David: The other thing that people have to understand is that sales velocity is a really important component to figuring out how many reps you should be hiring, whether they should be inside reps or field reps. If it takes you 90 days to close a deal, that can’t be a $5,000 a year deal, that has to be a $50,000 or even $150,000 a year deal.
Das: Kristina, I know you’ve done a lot of work with metrics. So how do those play in?
Kristina: Probably the one way to sum it all together is how many months does it take to pay back customer acquisition cost. Very commonly within the SaaS world, we talk about a 12-month CAC payback. We typically want to see for every dollar you spend on sales and marketing, you get a dollar back within a year. That means you can tweak the inputs any way you want. Let’s say that doing paid acquisition is really effective for you. Then, you can spend proportionally more on paid acquisition and less on sales reps. Vice versa, if you have a great inbound engine, you actually can hire a lot more sales reps and spend more on sales headcount. With all formulas, it’s a guide rail, so if you have customers that retain really, really well, let’s say you’re selling to the enterprise, and you’ve got a 90% or 95% annual retention rate, then your CAC payback could be between 12 and 24 months. But let’s say you’re selling to the SMB and churn is 2% or 3% monthly, which ends up being like 80% to 90% annual retention. Then, because your customer is less sticky, I would recommend looking at a CAC payback of 6 to 12 months.
Das: How should you think about doing a free trial versus a paid trial?
David: On the one hand, the bottoms up motion where people can try essentially a full version of a product before they buy it is extremely powerful. On the other hand, I’ve started to try to think about how I advise companies, when they are thinking about a free trial for something that might cost $100,000 or $200,000 a year? Do we do a paid pilot that has some sort of contractual obligation that if we meet then turns into a commercial engagement?
Kristina: I do think the beauty of the bottoms up business is that you can get people to try the entire experience of the product for free, and they fall in love with it, and a certain percentage will convert. And that works really, really well for products that can self-serve. When you start moving up market to more complex products, the challenge with trials is it takes work to actually implement the product, whether it be integrations, IT has to give access, etc. You lose that self-serve ability, which is so amazing in the trial. And so, I tend to be more in the camp of paid trials, if it costs you money to actually deploy the trial. And when you’re selling to bigger customers, they associate value when they have to pay. Once a customer has to pay you, then they feel a need to make the project successful and thus they will onboard, schedule things, give you data and access.
David: If you can get to a point where you get the customer to do that paid pilot, such that the only difference between a pilot and an actual customer is just the signing of a contract, that’s very powerful. Now, that does force you to have a really good pre-sales motion to make sure that you can deliver on the promise you’ve made your customers. When companies don’t have a great product, and they paper over it with professional services and sales engineering and post-sales support, that paid pilot thing doesn’t work because the experience isn’t good enough. So, it really is incumbent on the SaaS company that does a paid pilot to make sure that they are able to deliver on that experience.
Kristina: And one emerging trend recently is people signing an annual contract with a one or three month out, as a replacement to the paid pilot. Because it’s the best of both worlds, the SaaS company that’s selling the product gets a higher level of commitment. And the customer gets the optionality of opting out in the same way as a trial without any clawback. It really comes down to where procurement falls. Sometimes procurement is at the beginning of that decision, which makes it more like an annual contract. Sometimes procurement is at the one or three month opt-out period, which means the customer already has a great experience, loves the product, and it is an easier way to convert procurements to actually sign on…
David: And that is a really good segue into renewals. I always tell founders, you might have this subscription business, but it’s not a recurring revenue business until the second year when the revenue actually recurs. I think you really have the first three months to get a customer up and running and happy. And if they’re not, you then have about three months to fix it. And if all that works out, then the remaining six months of the contract can be focused on upsell and expansion.
Das: Awesome. Thank you, Kristina. Thank you, David.
Kristina: Thanks so much for having us. This was fun.
David: Yeah, a lot of fun, great topics, and our favorite thing to talk about.
'''

In [45]:
length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(text)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
summ_input = []
for input in inputs:
  output = model.generate(**input)
  summ_input.append(tokenizer.decode(*output, skip_special_tokens=True))
summ_new = ''
for i in range(len(summ_input)):
  try:
    text = summ_input[i]
    summ_new = summ_new + ' ' + text
  except:
    print(f'Broke for id {i}')
summ_new = summ_new.strip()

length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(summ_new)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
for input in inputs:
  output = model.generate(**input)
  final = tokenizer.decode(*output, skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 62 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [6]:
regex='(\S*)[:]'

In [7]:
data=re.findall(regex, text)
print(data)

['Das', 'David', 'Das', 'Kristina', 'David', 'Kristina', 'Das', 'Kristina', 'Das', 'Kristina', 'David', 'Das', 'Kristina', 'Das', 'David', 'Kristina', 'Das', 'Kristina', 'David', 'Kristina', 'David', 'Kristina', 'David', 'Kristina', 'Das', 'Kristina', 'David', 'Das', 'David', 'Kristina', 'Das', 'David', 'Kristina', 'Das', 'David', 'Kristina', 'David', 'Das', 'Kristina', 'Das', 'David', 'Kristina', 'David', 'Kristina', 'David', 'Das', 'Kristina', 'David']


In [8]:
data = list(set(data))

In [9]:
text

'\nDas: Hi and welcome to the a16z podcast. I’m Das, and in this episode, I talk SaaS go-to-market with David Ulevitch and our newest enterprise general partner Kristina Shen. The first half of the podcast looks at how remote work impacts the SaaS go-to-market and what the smartest founders are doing to survive the current crisis. The second half covers pricing approaches and strategy, including how to think about free versus paid trials and navigating the transition to larger accounts. But we start with why it’s easier to move upmarket than down… and the advantage that gives a SaaS startup against incumbents.\nDavid: If you have a cohort of customers that are paying you $10,000 a year for your product, you’re going to find a customer that self-selects and is willing to pay $100,000 a year. Once you get one of those, your organization will figure out how you sell to, how you satisfy and support, customers at that price point and that size. But it’s really hard for a company that sells 

In [10]:
lines = text.splitlines( )

In [11]:
lines[48].split()[0]

'David:'

In [12]:
len(lines)

49

In [13]:
das = []
david = []
kristina = []

In [14]:
for i in range(1, 48):
  if lines[i].split()[0] == 'Das' +':':
    das.append(lines[i])
  elif lines[i].split()[0] == 'David' +':':
    david.append(lines[i])
  elif lines[i].split()[0] == 'Kristina' +':':
    kristina.append(lines[i])

In [15]:
kristina

['Kristina: It’s culturally very, very hard to give a product away for free that you’re already charging for. It feels like you’re eating away at your own potential revenue when you do it. So most people who try it end up pulling back very quickly.',
 'Kristina: I think another classic example is Dropbox versus Box. Both started as bottoms up businesses, try before you buy. But Box quickly found, “Hey, I’d rather sell to IT.” And Dropbox said, “Hey, we’ve got a great freemium motion going.” And they catalyzed their business around referrals and giving away free storage and shared storage in a way that really helped drive their bottoms up business.',
 'Kristina: Don’t try to go from a 100-person company to a 20,000-person company. Start targeting early adopters, maybe they’re late stage pre-IPO companies, then newly IPO’d companies. Starting in tech tends to be a little bit easier because they tend to be early adopters. Going vertical by vertical can be a great strategy as well. Targeti

In [16]:
dastext = ''
for i in das:
  dastext = dastext + i + ' '
dastext = dastext.replace('Das:', '')
dastext = dastext.strip()
dastext

'Hi and welcome to the a16z podcast. I’m Das, and in this episode, I talk SaaS go-to-market with David Ulevitch and our newest enterprise general partner Kristina Shen. The first half of the podcast looks at how remote work impacts the SaaS go-to-market and what the smartest founders are doing to survive the current crisis. The second half covers pricing approaches and strategy, including how to think about free versus paid trials and navigating the transition to larger accounts. But we start with why it’s easier to move upmarket than down… and the advantage that gives a SaaS startup against incumbents.  Does that mean that it’s easier for a company to do this go-to-market if they’re a new startup as opposed to if they’re a pre-existing SaaS?  It’s a big leap to go from selling to smaller customers to larger customers. How have you seen SaaS companies know or get the timing right on that? Especially since it does seem like that’s really related to scaling your sales force?  So right no

In [17]:
davidtext = ''
for i in david:
  davidtext = davidtext + i + ' '
davidtext = davidtext.replace('David:', '')
davidtext = davidtext.strip()
davidtext

'If you have a cohort of customers that are paying you $10,000 a year for your product, you’re going to find a customer that self-selects and is willing to pay $100,000 a year. Once you get one of those, your organization will figure out how you sell to, how you satisfy and support, customers at that price point and that size. But it’s really hard for a company that sells up market to move down market, because they’ve already baked in all that expensive, heavy lifting sales motion. And so as you go down market with a lower price point, usually, you can’t actually support it.  This is actually one of the key reasons why the bottoms up SaaS motion is just so competitive, and compelling, and so destructive against the traditional sales-driven test motion. If you have that great product and people are choosing to use it, it’s very hard for somebody with a sales-driven motion, and all the cost that’s loaded into that, to be able to compete against it. There are so many markets where initial

In [18]:
kristinatext = ''
for i in kristina:
  kristinatext = kristinatext + i + ' '
kristinatext = kristinatext.replace('Kristina:', '')
kristinatext = kristinatext.strip()
kristinatext

'It’s culturally very, very hard to give a product away for free that you’re already charging for. It feels like you’re eating away at your own potential revenue when you do it. So most people who try it end up pulling back very quickly.  I think another classic example is Dropbox versus Box. Both started as bottoms up businesses, try before you buy. But Box quickly found, “Hey, I’d rather sell to IT.” And Dropbox said, “Hey, we’ve got a great freemium motion going.” And they catalyzed their business around referrals and giving away free storage and shared storage in a way that really helped drive their bottoms up business.  Don’t try to go from a 100-person company to a 20,000-person company. Start targeting early adopters, maybe they’re late stage pre-IPO companies, then newly IPO’d companies. Starting in tech tends to be a little bit easier because they tend to be early adopters. Going vertical by vertical can be a great strategy as well. Targeting one customer who might be branded 

In [19]:
summ_mod = "knkarthick/MEETING_SUMMARY"
sent_mod = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(summ_mod)
model = AutoModelForSeq2SeqLM.from_pretrained(summ_mod)

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

In [20]:
maxtokens = tokenizer.max_len_single_sentence

In [38]:
tokenizer = AutoTokenizer.from_pretrained(summ_mod)
model = AutoModelForSeq2SeqLM.from_pretrained(summ_mod)

In [21]:
length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(dastext)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)

1

In [22]:
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]

In [23]:
summdas_input = []
sentdas_input = []
for input in inputs:
  output = model.generate(**input)
  summdas_input.append(tokenizer.decode(*output, skip_special_tokens=True))

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 62 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [24]:
summdas_input

['In this episode of the a16z podcast, Das, David Ulevitch and Kristina Shen talk about the go-to-market of a software as a service (SaaS) company. They also talk about pricing and packaging of SaaS products.']

In [25]:
summdas_new = ''
for i in range(len(summdas_input)):
  try:
    text = summdas_input[i]
    summdas_new = summdas_new + ' ' + text
  except:
    print(f'Broke for id {i}')

In [26]:
summdas_new = summdas_new.strip()

In [27]:
summdas_new

'In this episode of the a16z podcast, Das, David Ulevitch and Kristina Shen talk about the go-to-market of a software as a service (SaaS) company. They also talk about pricing and packaging of SaaS products.'

In [28]:
length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(summdas_new)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)

1

In [29]:
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
for input in inputs:
  output = model.generate(**input)
  finaldas = tokenizer.decode(*output, skip_special_tokens=True)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 62 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [30]:
# David
length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(davidtext)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
summdav_input = []
sentdav_input = []
for input in inputs:
  output = model.generate(**input)
  summdav_input.append(tokenizer.decode(*output, skip_special_tokens=True))
summdav_new = ''
for i in range(len(summdav_input)):
  try:
    text = summdav_input[i]
    summdav_new = summdav_new + ' ' + text
  except:
    print(f'Broke for id {i}')
summdav_new = summdav_new.strip()

length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(summdav_new)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
for input in inputs:
  output = model.generate(**input)
  finaldav = tokenizer.decode(*output, skip_special_tokens=True)

In [31]:
# Kristina
length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(kristinatext)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
summkri_input = []
sentkri_input = []
for input in inputs:
  output = model.generate(**input)
  summkri_input.append(tokenizer.decode(*output, skip_special_tokens=True))
summkri_new = ''
for i in range(len(summkri_input)):
  try:
    text = summkri_input[i]
    summkri_new = summkri_new + ' ' + text
  except:
    print(f'Broke for id {i}')
summkri_new = summkri_new.strip()

length = 0
comb = ''
comblist = []
iter = 0
sentences = nltk.tokenize.sent_tokenize(summkri_new)
for sentence in sentences:
  iter+= 1
  total_len = len(tokenizer.tokenize(sentence)) + length
  if total_len < maxtokens:
    comb += sentence
    length = total_len
    if iter == len(sentences):
      comblist.append(comb.strip())
  else:
    comblist.append(comb.strip())
    comb = sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(comblist)
inputs = [tokenizer(comb, return_tensors='pt') for comb in comblist]
for input in inputs:
  output = model.generate(**input)
  finalkri = tokenizer.decode(*output, skip_special_tokens=True)

In [32]:
tokenizer = AutoTokenizer.from_pretrained(sent_mod)
model = AutoModelForSequenceClassification.from_pretrained(sent_mod)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [33]:
# Das
RobertaPosdas = []
RobertaNegdas = []
RobertaNeudas = []
for i in range(len(summdas_input)):
    text = summdas_input[i]
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    RobertaPosdas.append(scores[2])
    RobertaNegdas.append(scores[0])
    RobertaNeudas.append(scores[1])
RobertaPositivedas = np.mean(RobertaPosdas)
RobertaNegativedas = np.mean(RobertaNegdas)
RobertaNeutraldas = np.mean(RobertaNeudas)

In [34]:
# David
RobertaPosdav = []
RobertaNegdav = []
RobertaNeudav = []
for i in range(len(summdav_input)):
    text = summdav_input[i]
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    RobertaPosdav.append(scores[2])
    RobertaNegdav.append(scores[0])
    RobertaNeudav.append(scores[1])
RobertaPositivedav = np.mean(RobertaPosdav)
RobertaNegativedav = np.mean(RobertaNegdav)
RobertaNeutraldav = np.mean(RobertaNeudav)

In [35]:
# Kristina
RobertaPoskri = []
RobertaNegkri = []
RobertaNeukri = []
for i in range(len(summkri_input)):
    text = summkri_input[i]
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    RobertaPoskri.append(scores[2])
    RobertaNegkri.append(scores[0])
    RobertaNeukri.append(scores[1])
RobertaPositivekri = np.mean(RobertaPoskri)
RobertaNegativekri = np.mean(RobertaNegkri)
RobertaNeutralkri = np.mean(RobertaNeukri)

In [47]:
print(f'Headline: {final}\nDas:\nTalking Points: {finaldas}\nSentiment Analysis:\n  Positive: {RobertaPositivedas}\n  Negative: {RobertaNegativedas}\n  Neutral: {RobertaNeutraldas}\n-----\nDavid:\nTalking Points: {finaldav}\nSentiment Analysis:\n  Positive: {RobertaPositivedav}\n  Negative: {RobertaNegativedav}\n  Neutral: {RobertaNeutraldav}\n-----\nKristina:\nTalking Points: {finalkri}\nSentiment Analysis:\n  Positive: {RobertaPositivekri}\n  Negative: {RobertaNegativekri}\n  Neutral: {RobertaNeutralkri}')

Headline: In this episode of the a16z podcast, Das talks about the go-to-market for software as a service (SaaS) with David Ulevitch and Kristina Shen. They discuss the advantages and disadvantages of selling to the right customers at the right price point.
Das:
Talking Points: In this episode of the a16z podcast, Das, David Ulevitch and Kristina Shen talk about the go-to-market of a software as a service (SaaS) company.
Sentiment Analysis:
  Positive: 0.15367624163627625
  Negative: 0.005664228927344084
  Neutral: 0.8406593799591064
-----
David:
Talking Points: Kristina explains why the bottoms up SaaS business model is destructive against the traditional sales-driven model.
Sentiment Analysis:
  Positive: 0.20659105479717255
  Negative: 0.35353437066078186
  Neutral: 0.439874529838562
-----
Kristina:
Talking Points: It’s culturally very hard to give a product away for free. Box started as a bottoms up business and they catalyzed their business around referrals and giving away free st